# MLflow Tutorial
1. Experiment Tracking
2. Model Registry
3. Centralized mlflow repository using dagshub

**Note :**
- we log parameters, metric

# 1. Experiment Tracking

In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

print(np.unique(y, return_counts=True))

(array([0, 1]), array([900, 100]))


In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

### 1.1 Experiment-1 : Train Logistic Regression Classifier

In [4]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



### 1.2 Experiment-2: Train Random Forest Classifier

In [5]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       270
           1       0.95      0.67      0.78        30

    accuracy                           0.96       300
   macro avg       0.96      0.83      0.88       300
weighted avg       0.96      0.96      0.96       300



### 1.3 Experiment-3 : Train XGBoost

In [6]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



### 1.4 Experiment-4: Handle class imbalance using SMOTETomek and then Train XGBoost

In [7]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
print(np.unique(y_train_res, return_counts=True))

(array([0, 1]), array([619, 619]))


In [8]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



### 1.5 Track Experiments using mlflow

In [4]:
models = [
    ("Logistic Regression", LogisticRegression(C=1, solver='liblinear'),(X_train, y_train), (X_test, y_test)),
    ("Random Forest", RandomForestClassifier(n_estimators=30, max_depth=3), (X_train, y_train), (X_test, y_test)),
    ("XGB Classifier", XGBClassifier(use_label_encoder=False, eval_metric='logloss'), (X_train, y_train), (X_test, y_test)),
    ("XGBClassifier with SMOTE", XGBClassifier(use_label_encoder=False, eval_metric='logloss'), (X_train, y_train), (X_test, y_test))
]

In [5]:
reports = []
for model_name, model, train_set, test_set in models:
    X_train, y_train = train_set[0], train_set[1]
    X_test, y_test = test_set[0], test_set[1]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

print(reports)

[{'0': {'precision': 0.9454545454545454, 'recall': 0.9629629629629629, 'f1-score': 0.9541284403669725, 'support': 270.0}, '1': {'precision': 0.6, 'recall': 0.5, 'f1-score': 0.5454545454545454, 'support': 30.0}, 'accuracy': 0.9166666666666666, 'macro avg': {'precision': 0.7727272727272727, 'recall': 0.7314814814814814, 'f1-score': 0.749791492910759, 'support': 300.0}, 'weighted avg': {'precision': 0.9109090909090909, 'recall': 0.9166666666666666, 'f1-score': 0.91326105087573, 'support': 300.0}}, {'0': {'precision': 0.96415770609319, 'recall': 0.9962962962962963, 'f1-score': 0.9799635701275046, 'support': 270.0}, '1': {'precision': 0.9523809523809523, 'recall': 0.6666666666666666, 'f1-score': 0.7843137254901961, 'support': 30.0}, 'accuracy': 0.9633333333333334, 'macro avg': {'precision': 0.9582693292370712, 'recall': 0.8314814814814815, 'f1-score': 0.8821386478088503, 'support': 300.0}, 'weighted avg': {'precision': 0.9629800307219661, 'recall': 0.9633333333333334, 'f1-score': 0.96039858

In [6]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

# initialize mlflow
mlflow.set_experiment('Anomaly Detection')
mlflow.set_tracking_uri('http://localhost:5000')

2024/08/21 21:59:33 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detection' does not exist. Creating a new experiment.


In [10]:
import mlflow
from pathlib import Path

# Create an experiment name, which must be unique and case sensitive
experiment_id = mlflow.create_experiment(
    "Social NLP Experiments",
    artifact_location=Path.cwd().joinpath("mlruns").as_uri(),
    tags={"version": "v1", "priority": "P1"},
)
experiment = mlflow.get_experiment(experiment_id)
print(experiment_id)
print(experiment)

MlflowException: API request to endpoint /api/2.0/mlflow/experiments/create failed with error code 403 != 200. Response body: ''

In [7]:
print(mlflow.get_tracking_uri())

http://localhost:5000


In [21]:
for i, elem in enumerate(models):
    model_name = elem[0]
    model = elem[1]
    report = reports[i]

    with mlflow.start_run(run_name=model_name):
        mlflow.log_param('model', model_name)
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_class_1', report['1']['recall'])
        mlflow.log_metric('recall_class_0', report['0']['recall'])
        mlflow.log_metric('f1-score-macro', report['macro avg']['f1-score'])

        if 'XGB' in model_name:
            print(model_name)
            mlflow.xgboost.log_model(model, 'model')
        else:
            mlflow.sklearn.load_model(model, 'model')

2024/08/21 01:31:50 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/678590062160524926/runs/14c82f7dfcd14a7b96ae47a6598357bf.
2024/08/21 01:31:50 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/678590062160524926.


TypeError: stat: path should be string, bytes, os.PathLike or integer, not LogisticRegression

In [10]:
!mlflow gc

Traceback (most recent call last):
  File "/Users/premkumar/miniforge3/envs/venv_mlops/bin/mlflow", line 8, in <module>
    sys.exit(cli())
             ^^^^^
  File "/Users/premkumar/miniforge3/envs/venv_mlops/lib/python3.12/site-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/premkumar/miniforge3/envs/venv_mlops/lib/python3.12/site-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "/Users/premkumar/miniforge3/envs/venv_mlops/lib/python3.12/site-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/premkumar/miniforge3/envs/venv_mlops/lib/python3.12/site-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/premk